In [546]:
# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

#Imports pickle
import pickle

#Library for indicators
from finta import TA

#Another library for indicators
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.trend import PSARIndicator
from ta.trend import SMAIndicator
from ta.trend import macd
from ta.momentum import rsi
from ta.volume import OnBalanceVolumeIndicator

#Import yfinance
import yfinance as yf

#Import yfinance
import tensorflow as tf


In [ ]:
def 

In [548]:
#Downloads info from Yahoo Finance
df = yf.download(tickers="ETH-EUR",
                            period="60d",
                            interval="15m",
                            auto_adjust=True)


#Sort values from past to present
df = df.reset_index(drop=True)

df = df.drop(columns=['Volume'])


#Indicators with TA
#Adds simple moving average
df["sma7"] = SMAIndicator(close=df["Open"], window=7, fillna=True).sma_indicator()
df["sma25"] = SMAIndicator(close=df["Open"], window=25, fillna=True).sma_indicator()
df["sma99"] = SMAIndicator(close=df["Open"], window=99, fillna=True).sma_indicator()

#Adds Bollinger Bands
indicator_bb = BollingerBands(close=df["Open"], window=20, window_dev=2)
df['bb_bbm'] = indicator_bb.bollinger_mavg()
df['bb_bbh'] = indicator_bb.bollinger_hband()
df['bb_bbl'] = indicator_bb.bollinger_lband()

#Adds PSAR Indicator
indicator_psar = PSARIndicator(high=df["Close"], low=df["Low"], close=df["Close"], step=0.02, max_step=2, fillna=True)
df['psar'] = indicator_psar.psar()

# Add Moving Average Convergence Divergence (MACD) indicator
df["MACD"] = macd(close=df["Close"], window_slow=26, window_fast=12, fillna=True)
    
# Add Relative Strength Index (RSI) indicator
df["RSI"] = rsi(close=df["Close"], window=14, fillna=True)

#Indicators using finta
#Adds simple moving average
df["sma40"] = TA.SMA(df, period = 40)

# Add Relative Strength Index (RSI) indicator
#df["OBV"] = TA.OBV(df)

#Drops empty spaces
df.fillna(0, inplace=True)

#Drops empty spaces
df.head(50)


df = df.drop(['Low','Close','High'], axis=1)


df

[*********************100%***********************]  1 of 1 completed


/Users/esteban/miniforge3/envs/tf_m1/lib/python3.8/site-packages/ta/trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
/Users/esteban/miniforge3/envs/tf_m1/lib/python3.8/site-packages/ta/trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_down = pd.Series(index=self._psar.index)


,Open,sma7,sma25,sma99,bb_bbm,bb_bbh,bb_bbl,psar,MACD,RSI,sma40
0,1135.459473,1135.459473,1135.459473,1135.459473,0.000000,0.000000,0.000000,1139.689819,0.000000,100.000000,0.000000
1,1139.599487,1137.529480,1137.529480,1137.529480,0.000000,0.000000,0.000000,1139.951660,0.020888,100.000000,0.000000
2,1139.963623,1138.340861,1138.340861,1138.340861,0.000000,0.000000,0.000000,1139.689819,-0.006572,30.795481,0.000000
3,1139.623169,1138.661438,1138.661438,1138.661438,0.000000,0.000000,0.000000,1139.200562,0.129975,81.303091,0.000000
4,1141.359253,1139.201001,1139.201001,1139.201001,0.000000,0.000000,0.000000,1139.200562,0.340293,87.711223,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
5729,1708.575195,1710.437692,1708.965991,1689.373423,1710.256537,1718.236987,1702.276087,1717.061201,1.637118,49.297355,1706.971558
5730,1706.754272,1708.731166,1709.070811,1689.712661,1710.231873,1718.252317,1702.211428,1716.394857,1.318537,49.281277,1707.238358
5731,1706.420654,1707.619071,1709.083428,1689.929094,1710.174518,1718.286234,1702.062802,1715.755166,0.674360,44.079313,1707.332587
5732,1701.981323,1705.759957,1708.897358,1690.045252,1709.594861,1718.287944,1700.901778,1715.141063,0.398938,47.785014,1707.433521


In [549]:
#Loads model

model = tf.keras.models.load_model('Classifier_NN_hourly_v2.0')

#Makes the prediction

y_pred = model.predict(df.iloc[-1:].to_numpy().reshape(1,-1))

#print(df.loc[234,:].to_numpy().reshape(1,-1))

print(y_pred.tolist())


df.iloc[-1:]

[[1.0]]


,Open,sma7,sma25,sma99,bb_bbm,bb_bbh,bb_bbl,psar,MACD,RSI,sma40
5733,1703.954956,1706.098476,1709.034658,1690.213415,1709.020929,1717.611368,1700.43049,1714.551524,0.120071,46.957228,1707.538531


In [135]:
df_train = df.sample(frac=0.7, random_state=0)
df_valid = df.drop(df_train.index)

max_ = df_train.max(axis=0)
min_ = df_train.min(axis=0)



df_train = (df_train - min_) / (max_ - min_)
df_valid = (df_valid - min_) / (max_ - min_)

X_train = df_train.drop('Position', axis=1)
X_valid = df_valid.drop('Position', axis=1)
#y_train = df_train['Position']
#y_valid = df_valid['Position']

KeyError: "['Position'] not found in axis"